AIDA CAPSTONE PROJECT BOLA-DADDYSON
CNN VERSION 1

In [ ]:
!pip install tensorflow==2.15.0 tensorflow-hub keras==2.15.0

In [ ]:
import zipfile
import os
!pip install patool
import patoolib

# Path to the uploaded zip file
zip_file_path = "C:/Users/BOLA OLATUNJI/Documents/ACADEMIC/SCHOOL/Canada/ACADEMIC SASK/FALL SEMESTER 2024/FRI PROJ 613 CAPSTONE/MODELS/CNN/AIDA CAPSTONEv3.rar"

# Extract all files to a directory
# Use patoolib to extract rar files
patoolib.extract_archive(zip_file_path, outdir="/content/unzipped_files2")

# List the extracted files
print(os.listdir("/content/unzipped_files2"))


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt

KeyboardInterrupt: 

In [ ]:
#Change dir to suit
base_dir = '/content/unzipped_files2/AIDA CAPSTONEv3'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [ ]:
validation_dir

In [ ]:
#for Image
img_height, img_width = 224, 224 #Image size to conform to transfer learning
batch_size = 32
NUM_CLASSES = 2

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    validation_split=0.1,
    subset="both",
    seed=123
)


test_ds = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size = (img_height, img_width),
    batch_size = batch_size
)


In [ ]:
class_names = ["Authorized", "Unauthorized"]

plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Path to your dataset
dataset_dir = "/content/unzipped_files/AIDA CAPSTONEv3"

# Set parameters
IMG_SIZE = (224, 224)  # MobileNetV2 input size
BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 0.001

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load training and validation datasets
train_generator = train_datagen.flow_from_directory(
    directory=f"{dataset_dir}/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'  # Binary classification
)

val_generator = val_datagen.flow_from_directory(
    directory=f"{dataset_dir}/validation",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Load MobileNetV2 pre-trained model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Pooling layer
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Prevent overfitting
output = Dense(1, activation='sigmoid')(x)  # Binary classification (0/1)

# Define the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=val_generator.samples // BATCH_SIZE
)



In [ ]:
model.evaluate(test_ds)

## Visualizing results of the training

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('./foo.png')
plt.show()

In [ ]:
import numpy

plt.figure(figsize=(10,10))
for images, labels in test_ds.take(1):
  classifications = model(images)
  # print(classifications)

  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    index = numpy.argmax(classifications[i])
    plt.title("Pred: " + class_names[index] + " | Real: " + class_names[labels[i]])

In [ ]:
#WEBCAM PREDICT
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
# Set up webcam capture
cap = cv2.VideoCapture(0)
print("Camera opened:", cap.isOpened())

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 224)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 224)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Preprocess the frame
    input_width, input_height = 224, 224  # Replace with your model's input size
    resized_frame = cv2.resize(frame, (input_width, input_height))
    rgb_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
    normalized_frame = rgb_frame / 255.0
    input_frame = np.expand_dims(normalized_frame, axis=0)

    # Model prediction
    prediction = model.predict(input_frame)
   

    confidence = np.max(prediction)
    if confidence > 0.3:  # Confidence threshold
        predicted_class = class_names[np.argmax(prediction)]
    else:
        predicted_class = "Unauthorized"

    # Display the prediction on the frame
    cv2.putText(frame, f"Prediction: {predicted_class}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow('Live Prediction', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

# Save and Load trained models

## Save as Keras .h5 model

Ref: https://github.com/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l07c01_saving_and_loading_models.ipynb

In [ ]:
'''import time

t = time.time()

export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras, include_optimizer=True)

!ls'''

## Load Keras .h5 model and use for prediction

We will now load the model we just saved into a new model called reloaded. We will need to provide the file path and the custom_objects parameter. This parameter tells keras how to load the hub.KerasLayer from the feature_extractor we used for transfer learning.

In [ ]:
'''import tensorflow_hub as hub # Import the tensorflow_hub module
import tensorflow as tf
reloaded = tf.keras.models.load_model(
  export_path_keras,
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()'''

In [ ]:
'''result_rs50 = model.evaluate(test_ds)
reloaded_result_rs50 = reloaded.evaluate(test_ds)'''

In [ ]:
'''#PREDICT IMAGE
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load the trained model
#model = keras.models.load_model("C:/Users/BOLA OLATUNJI/Documents/ACADEMIC/SCHOOL/Canada/ACADEMIC SASK/FALL SEMESTER 2024/FRI PROJ 613 CAPSTONE/MODELS/CNN/1732208580.h5")
    
# Class names for predictions
#class_names = ["Authorized", "Unauthorized"]


def preprocess_image(image_path, img_size=(224, 224)):
    """
    Preprocess the input image to match the input size of the CNN model.
    Args:
        image_path (str): Path to the image file.
        img_size (tuple): Target size for resizing the image (default is (640, 640)).
    Returns:
        np.array: Preprocessed image ready for prediction.
    """
    # Load the image using OpenCV
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Unable to load image from path: {image_path}")
    
    # Resize the image to the required size
    img = cv2.resize(img, img_size)
    
    # Normalize pixel values to the range [0, 1]
    img = img / 255.0
    
    # Expand dimensions to match the model's input shape (1, height, width, channels)
    img = np.expand_dims(img, axis=0)
    
    return img

def predict_image(image_path):
    """
    Predict the class of the input image using the trained CNN model.
    Args:
        image_path (str): Path to the image file.
    """
    try:
        # Preprocess the image
        processed_image = preprocess_image(image_path)
        
        # Perform prediction
        predictions = model.predict(processed_image)
        
        # Get the index of the highest predicted class
        predicted_class = np.argmax(predictions[0])
        
        # Get the class name
        class_name = class_names[predicted_class]
        
        # Get the confidence score
        confidence_score = predictions[0][predicted_class] * 100
        
        print(f"Predicted Class: {class_name}")
        print(f"Confidence Score: {confidence_score:.2f}%")
    except Exception as e:
        print(f"Error during prediction: {e}")

# Example usage
# Replace 'path_to_image.jpg' with the path to your test image
image_path = "C:/Users/BOLA OLATUNJI/Documents/ACADEMIC/SCHOOL/Canada/ACADEMIC SASK/FALL SEMESTER 2024/FRI PROJ 613 CAPSTONE/MODELS/CNN/TEST2/455718079_10229671083974917_4938839240535204253_n.jpg"
predict_image(image_path)

In [ ]:
#WEBCAM PREDICT
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
# Set up webcam capture
cap = cv2.VideoCapture(0)
print("Camera opened:", cap.isOpened())

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 224)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 224)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Preprocess the frame
    input_width, input_height = 224, 224  # Replace with your model's input size
    resized_frame = cv2.resize(frame, (input_width, input_height))
    rgb_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
    normalized_frame = rgb_frame / 255.0
    input_frame = np.expand_dims(normalized_frame, axis=0)

    # Model prediction
    prediction = model.predict(input_frame)
   

    confidence = np.max(prediction)
    if confidence > 0.5:  # Confidence threshold
        predicted_class = class_names[np.argmax(prediction)]
    else:
        predicted_class = "Unauthorized"

    # Display the prediction on the frame
    cv2.putText(frame, f"Prediction: {predicted_class}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow('Live Prediction', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
'''import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
# Open webcam or video
input_width, input_height = 224, 224  # Replace with your model's input size
resized_frame = cv2.resize(frame, (input_width, input_height))
rgb_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
normalized_frame = rgb_frame / 255.0
input_frame = np.expand_dims(normalized_frame, axis=0)

# Model prediction
prediction = model.predict(input_frame)
cap = cv2.VideoCapture(0)  # Change to a video path for video files
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference
    results = model(frame)

    # Display results
    annotated_frame = results[0].plot()
    cv2.imshow("prediction", annotated_frame)

    # Break on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
